In [ ]:
!pip install transformers==4.17
# !pip install optuna
!pip install datasets
!pip install torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 57.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48

In [ ]:
from transformers import Trainer, TrainingArguments, DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time
from torchinfo import summary
from tabulate import tabulate

**Dataset Utilization**

In [ ]:
splits = {'pharo_train': 'data/pharo_train-00000-of-00001.parquet', 'pharo_test': 'data/pharo_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_test"])

display(train_df.head())
display(test_df.head())
display(train_df.shape)
display(test_df.shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,BlAnchorRelativeToElement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,BlArrowheadSimpleArrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead. | BlArrowh...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,BlArrowheadSimpleArrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,BlArrowheadSimpleArrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,BlArrowheadTriangle,i am a triangular arrow head.,0,i am a triangular arrow head. | BlArrowheadTri...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,BlArrowheadExamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,BlArrowheadSimpleArrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,BlArrowheadSimpleArrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,BlArrowheadSimpleArrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,BlArrowheadTriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


(1298, 6)

(289, 6)

In [ ]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,blanchorrelativetoelement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,blarrowheadsimplearrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead | blarrowhe...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,blarrowheadsimplearrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,blarrowheadsimplearrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,blarrowheadtriangle,i am a triangular arrow head.,0,i am a triangular arrow head | blarrowheadtria...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,blarrowheadexamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,blarrowheadsimplearrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,blarrowheadsimplearrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,blarrowheadsimplearrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,blarrowheadtriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


In [ ]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=7)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [ ]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    # Compute per-category precision, recall, and F1
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)

    # Compute per-category accuracy
    accuracy_per_label = (preds == labels).mean(axis=0)

    # Convert results into a dictionary
    metrics = {
        'accuracy_per_label': accuracy_per_label.tolist(),
        'precision_per_label': precision.tolist(),
        'recall_per_label': recall.tolist(),
        'f1_per_label': f1.tolist()
    }

    # Compute overall accuracy (if needed)
    overall_accuracy = (preds == labels).all(axis=1).mean()
    metrics['overall_accuracy'] = overall_accuracy

    return metrics

In [ ]:
# def objective(trial):
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
#     batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
#     num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
#     weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

#     training_args = TrainingArguments(
#         output_dir="./results",
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size * 2,
#         warmup_steps=10,
#         weight_decay=weight_decay,
#         evaluation_strategy="epoch",
#         learning_rate=learning_rate,
#         gradient_accumulation_steps=2,
#         report_to="none",
#     )

#     trainer = OneHotTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         compute_metrics=compute_metrics,
#     )

#     trainer.train()
#     eval_results = trainer.evaluate()

#     return eval_results["eval_f1"]

In [ ]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=20)

# print("Best trial:")
# print(study.best_trial)

In [ ]:
# best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4 * 2,
    learning_rate=1.78e-05,
    weight_decay=0.083,
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)


trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1038
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 1300


Epoch,Training Loss,Validation Loss,Accuracy Per Label,Precision Per Label,Recall Per Label,F1 Per Label,Overall Accuracy
1,No log,0.345538,"[0.8538461538461538, 0.8307692307692308, 0.8692307692307693, 0.9576923076923077, 0.8807692307692307, 0.823076923076923, 0.926923076923077]","[0.0, 0.872093023255814, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.6944444444444444, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.7731958762886598, 0.0, 0.0, 0.0, 0.0, 0.0]",0.288462
2,No log,0.266702,"[0.8538461538461538, 0.9230769230769231, 0.8615384615384616, 0.9576923076923077, 0.9346153846153846, 0.823076923076923, 0.926923076923077]","[0.0, 0.8928571428571429, 0.46875, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.9259259259259259, 0.4411764705882353, 0.0, 0.45161290322580644, 0.0, 0.0]","[0.0, 0.9090909090909091, 0.45454545454545453, 0.0, 0.6222222222222222, 0.0, 0.0]",0.438462
3,No log,0.218349,"[0.9, 0.9423076923076923, 0.8961538461538462, 0.9576923076923077, 0.9653846153846154, 0.8846153846153846, 0.926923076923077]","[1.0, 0.9345794392523364, 0.6, 0.0, 0.8666666666666667, 0.9, 0.0]","[0.3157894736842105, 0.9259259259259259, 0.6176470588235294, 0.0, 0.8387096774193549, 0.391304347826087, 0.0]","[0.48, 0.9302325581395349, 0.6086956521739131, 0.0, 0.8524590163934426, 0.5454545454545454, 0.0]",0.596154
4,0.297100,0.201939,"[0.9192307692307692, 0.9346153846153846, 0.9076923076923077, 0.9576923076923077, 0.9653846153846154, 0.9192307692307692, 0.926923076923077]","[1.0, 0.8888888888888888, 0.65625, 0.0, 0.8666666666666667, 0.9310344827586207, 0.0]","[0.4473684210526316, 0.9629629629629629, 0.6176470588235294, 0.0, 0.8387096774193549, 0.5869565217391305, 0.0]","[0.6181818181818182, 0.9244444444444444, 0.6363636363636364, 0.0, 0.8524590163934426, 0.72, 0.0]",0.676923
5,0.297100,0.183806,"[0.926923076923077, 0.9384615384615385, 0.8846153846153846, 0.9576923076923077, 0.9615384615384616, 0.9346153846153846, 0.926923076923077]","[1.0, 0.9339622641509434, 0.5434782608695652, 0.0, 0.8181818181818182, 0.9142857142857143, 0.0]","[0.5, 0.9166666666666666, 0.7352941176470589, 0.0, 0.8709677419354839, 0.6956521739130435, 0.0]","[0.6666666666666666, 0.9252336448598131, 0.625, 0.0, 0.84375, 0.7901234567901234, 0.0]",0.688462
6,0.297100,0.176910,"[0.9346153846153846, 0.9423076923076923, 0.8884615384615384, 0.9615384615384616, 0.9653846153846154, 0.9076923076923077, 0.9307692307692308]","[0.8620689655172413, 0.9514563106796117, 0.5555555555555556, 1.0, 0.8666666666666667, 0.7619047619047619, 1.0]","[0.6578947368421053, 0.9074074074074074, 0.7352941176470589, 0.09090909090909091, 0.8387096774193549, 0.6956521739130435, 0.05263157894736842]","[0.746268656716418, 0.9289099526066351, 0.6329113924050633, 0.16666666666666666, 0.8524590163934426, 0.7272727272727273, 0.1]",0.707692
7,0.297100,0.176984,"[0.9423076923076923, 0.9384615384615385, 0.8807692307692307, 0.9615384615384616, 0.9769230769230769, 0.9307692307692308, 0.9307692307692308]","[0.896551724137931, 0.9107142857142857, 0.5319148936170213, 1.0, 0.9032258064516129, 0.9117647058823529, 1.0]","[0.6842105263157895, 0.9444444444444444, 0.7352941176470589, 0.09090909090909091, 0.9032258064516129, 0.6739130434782609, 0.05263157894736842]","[0.7761194029850746, 0.9272727272727272, 0.6172839506172839, 0.16666666666666666, 0.9032258064516129, 0.775, 0.1]",0.719231
8,0.118200,0.176868,"[0.9423076923076923, 0.9307692307692308, 0.8846153846153846, 0.9615384615384616, 0.9730769230769231, 0.9230769230769231, 0.9307692307692308]","[0.8484848484848485, 0.9166666666666666, 0.5416666666666666, 1.0, 0.875, 0.8421052631578947, 0.6666666666666666]","[0.7368421052631579, 0.9166666666666666, 0.7647058823529411, 0.09090909090909091, 0.9032258064516129, 0.6956521739130435, 0.10526315789473684]","[0.7887323943661971, 0.9166666666666666, 0.6341463414634146, 0.16666666666666666, 0.8888888888888888, 0.7619047619047619, 0.18181818181818182]",0.715385
9,0.118200,0.182464,"[0.9423076923076923, 0.9307692307692308, 0.8884615384615384, 0.9615384615384616, 0.9769230769230769,

***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in label

TrainOutput(global_step=1300, training_loss=0.17775350864116962, metrics={'train_runtime': 284.6602, 'train_samples_per_second': 36.465, 'train_steps_per_second': 4.567, 'total_flos': 687567102474240.0, 'train_loss': 0.17775350864116962, 'epoch': 10.0})

In [ ]:
!mkdir 'best_model_distilbert_pharo'
!mkdir 'best_model_distilbert_pharo_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_distilbert_pharo')
tokenizer.save_pretrained('./best_model_distilbert_pharo_tokenizer')

# Load model and tokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('./best_model_distilbert_pharo')
tokenizer = DistilBertTokenizer.from_pretrained('./best_model_distilbert_pharo_tokenizer')

Saving model checkpoint to ./best_model_distilbert_pharo
Configuration saved in ./best_model_distilbert_pharo/config.json
Model weights saved in ./best_model_distilbert_pharo/pytorch_model.bin
tokenizer config file saved in ./best_model_distilbert_pharo_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_distilbert_pharo_tokenizer/special_tokens_map.json
loading configuration file ./best_model_distilbert_pharo/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "

## Model Testing

In [ ]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [ ]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [ ]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

# Display metrics and inference time
print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 289
  Batch size = 8


Evaluation Metrics: {'accuracy_per_label': [0.8858131487889274, 0.916955017301038, 0.8719723183391004, 0.986159169550173, 0.9653979238754326, 0.903114186851211, 0.9653979238754326], 'precision_per_label': [0.6470588235294118, 0.9130434782608695, 0.6595744680851063, 0.0, 0.8333333333333334, 0.6666666666666666, 0.0], 'recall_per_label': [0.5116279069767442, 0.8823529411764706, 0.5961538461538461, 0.0, 0.8333333333333334, 0.6976744186046512, 0.0], 'f1_per_label': [0.5714285714285714, 0.8974358974358975, 0.6262626262626263, 0.0, 0.8333333333333334, 0.6818181818181818, 0.0], 'overall_accuracy': 0.6851211072664359}
Inference Time: 2.37 seconds


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
!zip -r best_model_distilbert_pharo.zip './best_model_distilbert_pharo'
!zip -r best_model_distilbert_pharo_tokenizer.zip './best_model_distilbert_pharo_tokenizer'

# For downloading model locally
# from google.colab import files
# files.download('checkpoint-6000.zip')

  adding: best_model_distilbert_pharo/ (stored 0%)
  adding: best_model_distilbert_pharo/training_args.bin (deflated 51%)
  adding: best_model_distilbert_pharo/config.json (deflated 53%)
  adding: best_model_distilbert_pharo/pytorch_model.bin (deflated 8%)
  adding: best_model_distilbert_pharo_tokenizer/ (stored 0%)
  adding: best_model_distilbert_pharo_tokenizer/vocab.txt (deflated 53%)
  adding: best_model_distilbert_pharo_tokenizer/tokenizer_config.json (deflated 43%)
  adding: best_model_distilbert_pharo_tokenizer/special_tokens_map.json (deflated 40%)


In [ ]:
# prompt: transfer the checkpoint-6000.zip created to the google drive

from google.colab import drive
drive.mount('/content/drive')

!cp best_model_distilbert_pharo.zip "/content/drive/MyDrive/nlbse/models/DistilBert_pharo/"
!cp best_model_distilbert_pharo_tokenizer.zip "/content/drive/MyDrive/nlbse/models/DistilBert_pharo_Tokenizer/"

Mounted at /content/drive
cp: cannot create regular file '/content/drive/MyDrive/nlbse/models/DistilBert_pharo/': Not a directory
cp: cannot create regular file '/content/drive/MyDrive/nlbse/models/DistilBert_pharo_Tokenizer/': Not a directory


# **Load and Test Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/nlbse/models/DistilBert_pharo/'
tokenizer_folder = '/content/drive/MyDrive/nlbse/models/DistilBert_pharo_Tokenizer/'

!unzip '/content/drive/MyDrive/nlbse/models/DistilBert_pharo/best_model_distilbert_pharo.zip' -d './'
!unzip '/content/drive/MyDrive/nlbse/models/DistilBert_pharo_Tokenizer/best_model_distilbert_pharo_tokenizer.zip' -d './'

unzip:  cannot find or open /content/drive/MyDrive/nlbse/models/DistilBert_pharo/best_model_distilbert_pharo.zip, /content/drive/MyDrive/nlbse/models/DistilBert_pharo/best_model_distilbert_pharo.zip.zip or /content/drive/MyDrive/nlbse/models/DistilBert_pharo/best_model_distilbert_pharo.zip.ZIP.
unzip:  cannot find or open /content/drive/MyDrive/nlbse/models/DistilBert_pharo_Tokenizer/best_model_distilbert_pharo_tokenizer.zip, /content/drive/MyDrive/nlbse/models/DistilBert_pharo_Tokenizer/best_model_distilbert_pharo_tokenizer.zip.zip or /content/drive/MyDrive/nlbse/models/DistilBert_pharo_Tokenizer/best_model_distilbert_pharo_tokenizer.zip.ZIP.


In [ ]:
best_model_distilbert_pharo = 'best_model_distilbert_pharo'
best_model_distilbert_pharo_tokenizer = 'best_model_distilbert_pharo_tokenizer'

In [ ]:
# Load the model and tokenizer from the checkpoint
model = DistilBertForSequenceClassification.from_pretrained(best_model_distilbert_pharo)
tokenizer = DistilBertTokenizer.from_pretrained(best_model_distilbert_pharo_tokenizer)

text = "relative anchor takes an arbitrary element as"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

loading configuration file best_model_distilbert_pharo/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "multi_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "voca

SequenceClassifierOutput(loss=None, logits=tensor([[-1.2410, -4.9866,  0.4612, -4.2961, -4.0422, -2.5233, -3.4631]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 3


In [ ]:
# import torch

# # Logits from the model output
# logits =  output.logits #torch.tensor([[-17.5500, -13.8129, 9.7042, -16.6331, -6.6066]])

# # Get the index of the highest score
# predicted_class = ((torch.argmax(logits, dim=1).item()) + 1)

# print(f"Predicted class: {predicted_class}")
